In [ ]:
#!pip install seaborn

import pandas as pd
import ast
import seaborn as sns
import re
import matplotlib.pyplot as plt

In [ ]:
# Import cleaned dataset
df = pd.read_csv('../data/cleaned.csv')
len(df)

# Investigate Relationship Between Game Categories and Game Mechanics

In [ ]:
# Extract just the categories and mechanics columns for this analysis
cat_mech_df = df[['categories', 'mechanics']]\
    .copy()\
    .dropna()\
    .reset_index(drop=True)
cat_mech_df.info()

In [ ]:
# Group categories into broader ones

def group_war(categories):
    if isinstance(categories, str):
        listval = ast.literal_eval(categories)
    elif isinstance(categories, list):
        listval = categories
    listval = list(map(
        lambda x: re.sub('^.*War.*$', 'War', x),
        listval
    ))
    return listval

def group_value(values, old_value, new_value):
    if isinstance(values, str):
        listval = ast.literal_eval(values)
    elif isinstance(values, list):
        listval = values
    listval = list(map(
        lambda x: x.replace(old_value, new_value),
        listval
    ))
    return listval

cat_mech_df['categories'] = cat_mech_df['categories'].apply(
    group_war
)

category_mappings = [
    ('Historical Setting', 'American West'),
    ('Historical Setting', 'Ancient'),
    ('Historical Setting', 'Medieval'),
    ('Historical Setting', 'Napoleonic'),
    ('Historical Setting', 'Post-Napoleonic'),
    ('Historical Setting', 'Prehistoric'),
    ('Historical Setting', 'Renaissance'),
    ('Bluffing / Negotiation', 'Bluffing'),
    ('Bluffing / Negotiation', 'Negotiation'),
    ('Books', 'Book'),
    ('Books', 'Comic Book / Strip'),
    ('Books', 'Novel-based'),
    ('Economy / Industry', 'Economic'),
    ('Economy / Industry', 'Farming'),
    ('Economy / Industry', 'Industry / Manufacturing')
]

for mapping in category_mappings:
    cat_mech_df['categories'] = cat_mech_df['categories'].apply(
        lambda x: group_value(x, mapping[1], mapping[0])
    )

# Remove duplciates
cat_mech_df['categories'] = cat_mech_df['categories'].apply(
    lambda x: list(set(x))
)

cat_mech_df.head()

In [ ]:
# Group mechanics into broader ones

mechanics_mappings = [
    ('Auction', 'Auction: Once Around'),
    ('Auction', 'Closed Economy Auction'),
    ('Auction', 'Constrained Bidding'),
    ('Auction', 'Auction/Bidding'),
    ('Auction', 'Auction: Dutch Priority'),
    ('Auction', 'Auction: Dutch'),
    ('Auction', 'Auction: English'),
    ('Auction', 'Auction: Sealed Bid'),
    ('Auction', 'Auction: Turn Order Until Pass'),
    ('Auction', 'Auction: Multiple Lot'),
    ('Auction', 'Turn Order: Auction'),
    ('Auction', 'Auction Compensation'),
    ('Auction', 'Auction: Fixed Placement'),
    ('Auction', 'Auction: Dexterity'),
    ('Drafting', 'Action Drafting'),
    ('Drafting', 'Closed Drafting'),
    ('Drafting', 'Open Drafting'),
    ('Grid-Based', 'Hexagon Grid'),
    ('Grid-Based', 'Square Grid'),
    ('Grid-Based', 'Grid Movement')
]

for mapping in mechanics_mappings:
    cat_mech_df['mechanics'] = cat_mech_df['mechanics'].apply(
        lambda x: group_value(x, mapping[1], mapping[0])
    )

# Remove duplciates
cat_mech_df['mechanics'] = cat_mech_df['mechanics'].apply(
    lambda x: list(set(x))
)

cat_mech_df.head()

In [ ]:
# exploded_subset = cat_mech_df.copy()
exploded_subset = cat_mech_df\
    .copy()\
    .explode('categories')\
    .explode('mechanics')\
    .reset_index(drop=True)\
    .dropna()

# Create a matrix with categories on one axis, and mechanics on the other.
# Cell values are the number of occurrences.
exploded_subset['val'] = 1
matrix = exploded_subset.pivot_table(index='categories', columns='mechanics', values='val', aggfunc='sum', fill_value=0)
matrix
sns.heatmap(matrix)

## Impose a minimum threshold of 1500

Drop all rows/columns that don't have a max value of 1500+

In [ ]:
# Remove categories and mechanics that do not have a max value of at least 1500
threshold = 1500
matrix_filtered = matrix.copy()
for col in matrix_filtered.columns:
    if matrix_filtered[col].max() < threshold:
        matrix_filtered.drop([col], axis='columns', inplace=True)
for row in matrix_filtered.index:
    if matrix_filtered.loc[row].max() < threshold:
        matrix_filtered.drop([row], axis='index', inplace=True)

hm = sns.heatmap(matrix_filtered);
_ = hm.set_xticklabels(hm.get_xticklabels(), rotation=45, ha='right')
